# Color Image Quantization
Reduce the number of colors in images (`lena.png`, `peppers.tif`) to 16 using K-means clustering, implemented from scratch. Log L2 norms per iteration and optionally visualize quantized images.

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def kmeans_quantization(image, k=16, max_iterations=100, visualize=False):
    """Perform K-means color quantization on an image."""
    
    # 1. Reshape image to list of pixels (N, 3)
    h, w, c = image.shape
    pixels = image.reshape(-1, 3).astype(np.float32)   # (N, 3)
    n_pixels = pixels.shape[0]
    
    # 2. Initialize k centroids randomly from pixels
    rng = np.random.default_rng(seed=42)  # seed برای reproducibility
    centroids = pixels[rng.choice(n_pixels, size=k, replace=False)].copy()
    
    l2_norms = []
    
    for it in range(max_iterations):
        # 3. Assign pixels to nearest centroid (L2 norm)
        distances = np.linalg.norm(pixels[:, None] - centroids[None, :], axis=2)  # (N, k)
        labels = np.argmin(distances, axis=1)  # (N,)
        
        # 4. Update centroids as mean of assigned pixels
        new_centroids = np.copy(centroids)
        for j in range(k):
            mask = labels == j
            if np.any(mask):
                new_centroids[j] = pixels[mask].mean(axis=0)
        
        # 5. Create quantized image and calculate L2 norm
        quantized_pixels = new_centroids[labels]
        quantized_image = quantized_pixels.reshape(h, w, c)
        l2 = calculate_l2_norm(image, quantized_image)
        l2_norms.append(l2)
        
        print(f"Iteration {it+1}: L2 norm = {l2:.6f}")
        
        # Visualization
        if visualize and (it < 5 or it % 5 == 0): 
            plt.figure(figsize=(6, 6))
            plt.imshow(quantized_image)
            plt.title(f"Iteration {it+1}, L2={l2:.2f}")
            plt.axis("off")
            plt.show()
        
        # Check for convergence
        if np.allclose(new_centroids, centroids, atol=1e-6):
            print(f"Converged after {it+1} iterations")
            break
        
        centroids = new_centroids
    
    # Final quantized image
    final_quantized_pixels = centroids[labels]
    final_quantized_image = final_quantized_pixels.reshape(h, w, c)
    
    return final_quantized_image, l2_norms
          


In [13]:
def calculate_l2_norm(original, quantized):
    """Calculate total L2 norm between original and quantized images."""
    diff = original - quantized
    l2_norm = np.sqrt(np.sum(diff ** 2))  # Total L2 norm
    return l2_norm

In [23]:
# Load images
lena_path = './images/lena.png'

lena = cv2.imread(lena_path).astype(np.float32) / 255.0

# Parameters
k = 16
max_iterations = 100
visualize = True

In [ ]:
# Process images
lena_quantized, lena_l2_norms = kmeans_quantization(lena, k, max_iterations, visualize)

# Save L2 norms
with open('L2_norm_log.txt', 'w') as f:
    f.write('Lena L2 Norms:\n' + '\n'.join(map(str, lena_l2_norms)) + '\n')

# Calculate final L2 norms
lena_l2 = calculate_l2_norm(lena, lena_quantized)

# Visualize results
plt.imshow(cv2.cvtColor(lena_quantized, cv2.COLOR_BGR2RGB))
plt.title('Lena Quantized')
plt.axis('off')
plt.show()

print(f'Total L2 Norm: {lena_l2:.2f}')

# Save quantized images
cv2.imwrite('lena_quantized.png', (lena_quantized * 255).astype(np.uint8))